In [1]:
# Data
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
# import impl.utils as utils
import impl.layer as l

# Dataset preparation and pre-processing
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)

X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels
# y_test.shape, y_val.shape, y_train.shape

M, D, C = X_train.shape[0], X_train.shape[1], y_train.max() + 1
# M, D, C

X_train, X_val, X_test = l.prepro(X_train, X_val, X_test)
# X_train.shape, X_val.shape, X_test.shape
# if net_type == 'cnn':
img_shape = (1, 28, 28)

X_train = X_train.reshape(-1, *img_shape)
X_val = X_val.reshape(-1, *img_shape)
X_test = X_test.reshape(-1, *img_shape)
X_train.shape, X_val.shape, X_test.shape

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


((55000, 1, 28, 28), (5000, 1, 28, 28), (10000, 1, 28, 28))

In [2]:
# Model
# import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.NN as nn

class DCNN(nn.NN):

    def __init__(self, D, C, H, L, p_dropout, lam=1e-3, loss='cross_ent', nonlin='relu'):
        self.p_dropout = p_dropout
        self.loss = loss
        self.mode = 'classification'
        self.L = L # number of layers or depth
        super().__init__(D, C, H, lam, p_dropout, loss, nonlin)
        
    def _init_model(self, D, C, H):
        self.model = []
        
        # Input layer of Conv
        self.model.append(dict(
            W1=np.random.randn(H, 1, 3, 3) / np.sqrt(H / 2.),
            b1=np.zeros((H, 1)),
            W1_res=np.random.randn(H, 1, 3, 3) / np.sqrt(H / 2.),
            b1_res=np.zeros((H, 1))
        ))
        
        # Hidden layers of Conv-bn-relu-dropout
        m = []
        self.bn_caches = []
        for layer in range(self.L):
            m.append(dict(
                    W2=np.random.randn(H, H, 3, 3) / np.sqrt(H / 2.),
                    b2=np.zeros((H, 1)),
                    W2_res=np.random.randn(H, H, 3, 3) / np.sqrt(H / 2.),
                    b2_res=np.zeros((H, 1))
            ))
        self.model.append(m) # self.model[0][]
        
        # Output layer of FC to output
        self.model.append(dict(
            W3=np.random.randn(H*D, C) / np.sqrt(H*D / 2.),
            b3=np.zeros((1, C))
        ))

    def selu_forward(self, X):
        alpha = 1.6732632423543772848170429916717
        scale = 1.0507009873554804934193349852946
        out = scale * np.where(X>=0.0, X, alpha * (np.exp(X)-1))
        cache = X
        return out, cache

    def selu_backward(self, dout, cache):
        alpha = 1.6732632423543772848170429916717
        scale = 1.0507009873554804934193349852946
        X = cache
        dX_pos = dout.copy()
        dX_pos[X<0] = 0
        dX_neg = dout.copy()
        dX_neg[X>0] = 0
        dX = scale * np.where(X>=0.0, dX_pos, dX_neg * alpha * np.exp(X))
        return dX

    #     # """"""""""""""""""""""""""""""""""""""Dropout to a value with rescaling."""""""""""""""""""""""""""""""""""""""
    #     # NumPy implementation
    #     def dropout_selu_forward(self, X, p_dropout):
    #         #         alpha = 1.6732632423543772848170429916717
    #         #         scale = 1.0507009873554804934193349852946
    #         #         alpha * scale = 1.7580993408473766
    #         alpha= -1.7580993408473766
    #         fixedPointMean=0.0 
    #         fixedPointVar=1.0
    #         keep_prob = 1.0 - p_dropout
    #         #     noise_shape = noise_shape.reshape(X)
    #         random_tensor = keep_prob
    #         #         random_tensor += random_ops.random_uniform(noise_shape, seed=seed, dtype=x.dtype)
    #         random_tensor += np.random.uniform(size=X.shape) # low=0, high=1
    #         #         binary_tensor = math_ops.floor(random_tensor)
    #         binary_tensor = np.floor(random_tensor)
    #         #         ret = X * binary_tensor + alpha * (1-binary_tensor)
    #         X = X * binary_tensor + alpha * (1-binary_tensor)

    #         #         a = tf.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * tf.pow(alpha-fixedPointMean,2) + fixedPointVar)))
    #         a = np.sqrt(fixedPointVar / (keep_prob *((1-keep_prob) * ((alpha-fixedPointMean)**2) + fixedPointVar)))

    #         b = fixedPointMean - a * (keep_prob * fixedPointMean + (1 - keep_prob) * alpha)
    #         #         ret = a * ret + b
    #         out = a * X + b
    #         #         ret.set_shape(x.get_shape())
    #         #         ret = ret.reshape(X.shape)
    #         cache = a, binary_tensor
    #         return out, cache

    #     def dropout_selu_backward(self, dout, cache):
    #         a, binary_tensor = cache
    #         dX = dout * a
    #         dX = dX * binary_tensor
    #         return dX
    
    def alpha_dropout_fwd(self, h, q):
        '''h is activation, q is keep probability: q=1-p, p=p_dropout, and q=keep_prob'''
        alpha = 1.6732632423543772848170429916717
        scale = 1.0507009873554804934193349852946
        alpha_p = -scale * alpha
        mask = np.random.binomial(1, q, size=h.shape)
        dropped = mask * h + (1 - mask) * alpha_p
        a = 1. / np.sqrt(q + alpha_p ** 2 * q  * (1 - q))
        b = -a * (1 - q) * alpha_p
        out = a * dropped + b
        cache = (a, mask)
        return out, cache

    def alpha_dropout_bwd(self, dout, cache):
        a, mask = cache
        d_dropped = dout * a
        dh = d_dropped * mask
        return dh

    def forward(self, X, train=False):

        # 1st layer: Input to Conv
        h1_res, conv1_res_cache = l.conv_forward(X=X, W=self.model[0]['W1_res'], b=self.model[0]['b1_res']) 
        h1, conv1_cache = l.conv_forward(X=X, W=self.model[0]['W1'], b=self.model[0]['b1']) 
        #         h2, nl1_cache = l.relu_forward(h1)
        h2, nl1_cache = self.selu_forward(h1)
        do1_cache = None
        #         if train: h2, do1_cache = self.dropout_selu_forward(X=h2, p_dropout=self.p_dropout)
        if train: h2, do1_cache = self.alpha_dropout_fwd(h=h2, q=1.0-self.p_dropout)
        # x = x + f(x)
        h1_res += h2
        h1_cache = (conv1_res_cache, conv1_cache, nl1_cache, do1_cache)

        ###########################################################################################
        h2_cache = []
        for layer in range(self.L):
            # midst layer: Convnet 1
            if not layer == 0: h2 = h2.reshape(nl1_cache.shape)

            h2_res, conv2_res_cache = l.conv_forward(X=h2, W=self.model[1][layer]['W2_res'], 
                                                     b=self.model[1][layer]['b2_res'])
            h2_res = h2_res.reshape([nl1_cache.shape[0], -1])

            h2, conv2_cache = l.conv_forward(X=h2, W=self.model[1][layer]['W2'], b=self.model[1][layer]['b2'])
            h2 = h2.reshape([nl1_cache.shape[0], -1])
            
            #             h2, nl2_cache = l.relu_forward(h2)
            h2, nl2_cache = self.selu_forward(X=h2)
            do2_cache = None # ERROR: referenced before assigned!
            #             if train: h2, do2_cache = l.dropout_forward(X=h2, p_dropout=self.p_dropout)
            #             if train: h2, do2_cache = self.dropout_selu_forward(X=h2, p_dropout=self.p_dropout)
            if train: h2, do2_cache = self.alpha_dropout_fwd(h=h2, q=1.0-self.p_dropout)
            h2_res += h2    

            cache = (conv2_res_cache, conv2_cache, nl2_cache, do2_cache)
            h2_cache.append(cache)
        ############################################################################################
            
        # last layer : FC to Output
        h3, h3_cache = l.fc_forward(X=h2, W=self.model[2]['W3'], b=self.model[2]['b3'])

        cache = (h1_cache, h2_cache, h3_cache)
        return h3, cache

    def cross_entropy(self, y_pred, y_train):
        m = y_pred.shape[0]

        prob = l.softmax(y_pred)
        log_like = -np.log(prob[range(m), y_train])
        data_loss = np.sum(log_like) / m

        return data_loss

    def dcross_entropy(self, y_pred, y_train):
        m = y_pred.shape[0]

        grad_y = l.softmax(y_pred)
        grad_y[range(m), y_train] -= 1.0
        grad_y /= m

        return grad_y

    def loss_function(self, y, y_train):
        loss = self.cross_entropy(y, y_train)
        dy = self.dcross_entropy(y, y_train)
        return loss, dy
    
    def backward(self, dy, cache):
        h1_cache, h2_cache, h3_cache = cache
        conv1_res_cache, conv1_cache, nl1_cache, do1_cache = h1_cache

        # last layer
        dh2, dw3, db3 = l.fc_backward(dout=dy, cache=h3_cache)
        
        # midst layer 2
        g = []
        for layer in reversed(range(self.L)):
            conv2_res_cache, conv2_cache, nl2_cache, do2_cache = h2_cache[layer]
            dh2_res = dh2.reshape(nl1_cache.shape)
            dh2_res, dw2_res, db2_res = l.conv_backward(dout=dh2_res, cache=conv2_res_cache)
            #             dh2 = l.dropout_backward(dout=dh2, cache=do2_cache)
            #             dh2 = self.dropout_selu_backward(dout=dh2, cache=do2_cache)
            dh2 = self.alpha_dropout_bwd(dout=dh2, cache=do2_cache)
            #             dh2 = l.relu_backward(dout=dh2, cache=nl2_cache)
            dh2 = self.selu_backward(dout=dh2, cache=nl2_cache)
            dh2 = dh2.reshape(nl1_cache.shape)
            dh2, dw2, db2 = l.conv_backward(dout=dh2, cache=conv2_cache)
            dh2 += dh2_res
            if not layer==0: dh2 = dh2.reshape([nl1_cache.shape[0], -1])
            g.append(dict(
                    W2=dw2,
                    b2=db2,
                    W2_res=dw2_res,
                    b2_res=db2_res
                    ))
            
        # 1st layer
        #         conv1_res_cache, conv1_cache, nl1_cache, do1_cache = h1_cache
        dX_res, dw1_res, db1_res = l.conv_backward(dout=dh2, cache=conv1_res_cache)
        #         dh2 = self.dropout_selu_backward(dout=dh2, cache=do1_cache)
        dh2 = self.alpha_dropout_bwd(dout=dh2, cache=do1_cache)
        #         dh1 = l.relu_backward(dout=dh2, cache=nl1_cache)
        dh1 = self.selu_backward(dout=dh2, cache=nl1_cache)
        dX, dw1, db1 = l.conv_backward(dout=dh1, cache=conv1_cache)
        dX += dX_res

        # grad for GD
        grad = []
        
        # Input layer to conv layer
        grad.append(dict(
            W1=dw1, 
            b1=db1,
            W1_res=dw1_res, 
            b1_res=db1_res
        ))
        
        # Hidden layers of conv-bn-nl/relu-dropout/do
        grad.append(g)
        
        # Output later to FC layer
        grad.append(dict(
            W3=dw3, 
            b3=db3
        ))
        
        return dX, grad
    
    def test(self, X):
        y_logit, cache = self.forward(X, train=False)
        y_prob = l.softmax(y_logit)
        if self.mode == 'classification':
            return np.argmax(y_prob, axis=1)
        else: # self.mode == 'regression'
            return np.round(y_logit)

In [4]:
# SGD
# import numpy as np
# import impl.utils as util
# import impl.constant as c
# import copy
from sklearn.utils import shuffle as skshuffle

def get_minibatch(X, y, minibatch_size, shuffle=True):
    minibatches = []

    if shuffle:
        X, y = skshuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

def adam(nn, X_train, y_train, val_set, alpha, mb_size, n_iter, print_after):
    minibatches = get_minibatch(X_train, y_train, mb_size)

    if val_set:
        X_val, y_val = val_set
        
    M, R = [], []
    M.append({k: np.zeros_like(v) for k, v in nn.model[0].items()})
    R.append({k: np.zeros_like(v) for k, v in nn.model[0].items()})
    
    M_, R_ = [], []
    for layer in range(nn.L):
        M_.append({k: np.zeros_like(v) for k, v in nn.model[1][layer].items()})
        R_.append({k: np.zeros_like(v) for k, v in nn.model[1][layer].items()})
    M.append(M_)
    R.append(R_)

    M.append({k: np.zeros_like(v) for k, v in nn.model[2].items()})
    R.append({k: np.zeros_like(v) for k, v in nn.model[2].items()})
    beta1 = .99
    beta2 = .999
    #     import impl.constant as c, c.eps
    eps = 1e-8 # constant

    for iter in range(1, n_iter + 1):
        idx = np.random.randint(0, len(minibatches))
        X_mini, y_mini = minibatches[idx]

        #         grad, loss = nn.train_step(X_mini, y_mini)
        #         def train_step(self, X_train, y_train):
        #         """
        #         Single training step over minibatch: forward, loss, backprop
        #         """
        y, cache = nn.forward(X_mini, train=True)
        loss, dy = nn.loss_function(y, y_mini)
        dX, grad = nn.backward(dy, cache)
        #         return grad, loss

        if iter % print_after == 0:
            if val_set:
                val_acc = l.accuracy(y_val, nn.test(X_val))
                print('Iter-{} training loss: {:.4f} validation accuracy: {:4f}'.format(iter, loss, val_acc))

        for k in grad[0]:
            M[0][k] = l.exp_running_avg(M[0][k], grad[0][k], beta1)
            R[0][k] = l.exp_running_avg(R[0][k], grad[0][k]**2, beta2)

            m_k_hat = M[0][k] / (1. - (beta1**(iter)))
            r_k_hat = R[0][k] / (1. - (beta2**(iter)))

            nn.model[0][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + eps)
        
        for layer in range(nn.L):
            for k in grad[1][0]:
                M[1][layer][k] = l.exp_running_avg(M[1][layer][k], grad[1][layer][k], beta1)
                R[1][layer][k] = l.exp_running_avg(R[1][layer][k], grad[1][layer][k]**2, beta2)

                m_k_hat = M[1][layer][k] / (1. - (beta1**(iter)))
                r_k_hat = R[1][layer][k] / (1. - (beta2**(iter)))

                nn.model[1][layer][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + eps)
                        
        for k in grad[2]:
            M[2][k] = l.exp_running_avg(M[2][k], grad[2][k], beta1)
            R[2][k] = l.exp_running_avg(R[2][k], grad[2][k]**2, beta2)

            m_k_hat = M[2][k] / (1. - (beta1**(iter)))
            r_k_hat = R[2][k] / (1. - (beta2**(iter)))

            nn.model[2][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + eps)

    return nn

In [6]:
# Hyper-parameters
n_iter = 40 # number of epochs
alpha = 1e-3 # learning_rate
mb_size = 64 # width, timestep for sequential data or minibatch size
num_layers = 2 # depth 
print_after = 1 # print loss for train, valid, and test
num_hidden_units = 8
p_dropout = 0.05 #  keep_prob = 1.0 - p_dropout, q = 1-p, q=0.95, o=0.05

In [7]:
# build the model/NN and learn it: running session.
net = DCNN(C=C, D=D, H=num_layers, p_dropout=p_dropout, L=num_layers)

net = adam(nn=net, X_train=X_train, y_train=y_train, val_set=(X_val, y_val), mb_size=mb_size, alpha=alpha, 
           n_iter=n_iter, print_after=print_after)

y_pred = net.predict(X_test)
accs = np.mean(y_pred == y_test)

print()
print('Test Mean accuracy: {:.4f}, std: {:.4f}'.format(accs.mean(), accs.std()))

Iter-1 training loss: 26.9109 validation accuracy: 0.080400
Iter-2 training loss: 19.0822 validation accuracy: 0.120200
Iter-3 training loss: 15.1873 validation accuracy: 0.109400
Iter-4 training loss: 16.0085 validation accuracy: 0.077800
Iter-5 training loss: 18.9570 validation accuracy: 0.086000
Iter-6 training loss: 15.2785 validation accuracy: 0.108600
Iter-7 training loss: 11.2255 validation accuracy: 0.145800
Iter-8 training loss: 11.9566 validation accuracy: 0.194000
Iter-9 training loss: 9.9362 validation accuracy: 0.226000
Iter-10 training loss: 11.6925 validation accuracy: 0.264200
Iter-11 training loss: 10.5592 validation accuracy: 0.307800
Iter-12 training loss: 10.7979 validation accuracy: 0.332600
Iter-13 training loss: 8.3487 validation accuracy: 0.362200
Iter-14 training loss: 9.8731 validation accuracy: 0.401400
Iter-15 training loss: 9.4016 validation accuracy: 0.466800
Iter-16 training loss: 8.6683 validation accuracy: 0.514400
Iter-17 training loss: 10.0367 validat